<a href="https://colab.research.google.com/github/GitMishka/2021_TSLAstock_Analysis-/blob/master/yammer_knox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
#drive.mount('/content/drive/MyDrive/Yammer')
import pandas as pd
import datetime

In [ ]:
azure = pd.read_csv('/content/drive/MyDrive/Yammer/exportUsers_2023-5-18.csv')
yammer = pd.read_csv('/content/drive/MyDrive/Yammer/YammerActivityUserDetail5_18_2023 1_56_26 PM.csv')


In [ ]:
import pandas as pd
import datetime

def sanitize_sheet_name(name):
    name = str(name)
    invalid_chars = ['\\', '/', '*', '[', ']', ':', '?']
    for char in invalid_chars:
        name = name.replace(char, '')
    return name

def contains_knox(name):
    return 'knox' in str(name).lower()

def color_code(val):
    r_min, g_min, b_min = 255, 182, 193  # Light Pink
    r_max, g_max, b_max = 144, 238, 144  # Light Green
    ratio = val / 100
    r = r_min + int((r_max - r_min) * (1 - ratio))  # subtract ratio from 1 for descending values
    g = g_min + int((g_max - g_min) * ratio)
    b = b_min + int((b_max - b_min) * ratio)
    return f'background-color: rgb({r}, {g}, {b})'

yammer = yammer.rename(columns={'User Principal Name': 'userPrincipalName'})

merged_df = azure.merge(yammer, on='userPrincipalName')

result = merged_df[['userPrincipalName','Display Name', 'jobTitle', 'department', 'Posted Count', 'Read Count', 'Liked Count']]

unique_departments = result['department'].unique()
unique_principals = result['userPrincipalName'].unique()

knox_departments = [department for department in unique_departments if contains_knox(department)]
knox_principals = [principal for principal in unique_principals if contains_knox(principal)]

knox_entities = knox_departments + knox_principals

current_date = datetime.datetime.now().strftime('%Y-%m-%d')

file_name = f'Yammer_Results_{current_date}.xlsx'
with pd.ExcelWriter(file_name, engine='openpyxl') as writer:
    result.to_excel(writer, sheet_name='All Data', index=False)
    for entity in knox_entities:
        sanitized_entity = sanitize_sheet_name(entity)
        entity_data = result[(result['department'] == entity) | (result['userPrincipalName'] == entity)]
        #entity_data = entity_data.style.applymap(color_code, subset=['Posted Count', 'Read Count', 'Liked Count'])
        entity_data.to_excel(writer, sheet_name=f'{sanitized_entity}', index=False)

from google.colab import files
files.download(file_name)


/usr/local/lib/python3.10/dist-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>